In [2]:
# from distributed import Client
# client = Client()
import pandas as pd
# import modin.pandas as pd
import os
import json
from pathlib import Path
from typing import Optional
from urllib.request import urlretrieve
from tqdm import tqdm
from zipfile import ZipFile
from google.cloud import bigquery
from prefect import task, flow
from prefect_gcp.cloud_storage import GcsBucket
pd.set_option("display.max_columns", None)
print("Setup Complete")

Setup Complete


In [15]:
# Seq 1-Define a function to convert the downloaded file to data frame
def read_df(file: str) -> pd.DataFrame:
    with open(file) as data_file:
        data = json.load(data_file)
        df = pd.read_json(data)
        df = pd.json_normalize(df.to_dict("records"), sep="_")
        return df
    
file = "/Users/reneboygarcia/Downloads/albums-json/albums-full-info-3.json"
df = read_df(file)
df.head()

,_id,albumRelease,image,numTracks,keywords,datePublished,@id,@type,name,dateModified,@context,comment,description,inAlbum,offers,duration_secs,url,duration,recordingOf,isrcCode,@graph,byArtist_image,byArtist_genre,byArtist_@id,byArtist_@type,byArtist_sameAs,byArtist_name,byArtist_description,track_itemListElement,track_@type,track_numberOfItems,track,inAlbum_name,inAlbum_@id,inAlbum_@type,offers_availability,offers_priceSpecification_minPrice,offers_price,offers_@type,offers_priceCurrency,offers_url,recordingOf_@type,recordingOf_lyrics_text,recordingOf_lyrics_@type,byArtist
0,https://lukagluvic.bandcamp.com/album/cesta-ki...,[{'image': 'https://f4.bcbits.com/img/a3217976...,https://f4.bcbits.com/img/a3217976062_10.jpg,9.0,"Country, akustika, blues, folk, jazz, poezija,...",05 Aug 2020 11:49:47 GMT,https://lukagluvic.bandcamp.com/album/cesta-ki...,MusicAlbum,"Cesta, ki pelje v sanje",05 Aug 2020 11:49:47 GMT,https://schema.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://f4.bcbits.com/img/0020319935_10.jpg,https://bandcamp.com/tag/country,https://lukagluvic.bandcamp.com,MusicGroup,"[https://www.youtube.com/user/Lukag121, https:...",Luka Gluvić,\nSinger-songwriter working at the crossroads ...,"[{'position': 1, 'item': {'duration_secs': 135...",ItemList,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://unifactor.bandcamp.com/album/the-back-...,"[{'@type': ['Product', 'MusicRelease'], 'image...",https://f4.bcbits.com/img/a1986613900_10.jpg,6.0,"Experimental, cleveland, tapes, Cleveland",21 Dec 2017 16:15:12 GMT,https://unifactor.bandcamp.com/album/the-back-...,MusicAlbum,The Back of The Garden,18 Jun 2020 15:49:25 GMT,https://schema.org,[{'text': ['A burst of arpeggiated sunshine in...,The recording style alone on Brett Naucke's Th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://f4.bcbits.com/img/0020896591_10.jpg,https://bandcamp.com/tag/experimental,https://unifactor.bandcamp.com,MusicGroup,"[https://www.instagram.com/unifactor, https://...",Brett Naucke,NaN,"[{'@type': 'ListItem', 'position': 1, 'item': ...",ItemList,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://tinyfractures.bandcamp.com/album/perfe...,"[{'@type': ['Product', 'MusicRelease'], 'offer...",https://f4.bcbits.com/img/a3841284358_10.jpg,3.0,"Punk, emo, hardcore, sad, screamo, Boston",06 Apr 2015 23:48:49 GMT,https://tinyfractures.bandcamp.com/album/perfe...,MusicAlbum,Perfect Moments,20 Apr 2015 00:47:32 GMT,https://schema.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://bandcamp.com/tag/punk,https://tinyfractures.bandcamp.com,MusicGroup,NaN,Tiny Fractures,four people who write unfortunate pieces of music,"[{'@type': 'ListItem', 'position': 1, 'item': ...",ItemList,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://zjisin.bandcamp.com/track/seele-im-ges...,NaN,[https://f4.bcbits.com/img/a2600691313_10.jpg],NaN,"Electronic, analogue, dancefloor, poetry, Cologne",30 Jan 2019 10:42:02 GMT,https://zjisin.bandcamp.com/track/seele-im-ges...,"[MusicRecording, Product]",Seele Im Gesicht,30 Jan 2019 10:42:01 GMT,https://schema.org,NaN,NaN,NaN,NaN,323.5,https://zjisin.bandcamp.com/track/seele-im-ges...,P00H05M23S,NaN,NaN,NaN,https://f4.bcbits.com/img/0015434247_10.jpg,https://bandcamp.com/tag/electronic,https://zjisin.bandcamp.com,MusicGroup,NaN,Zjisin,NaN,NaN,NaN,NaN,NaN,"Heute Insta, Morgen Tod",https://zjisin.bandcamp.com/album/heute-insta-...,MusicAlbum,OnlineOnly,1.0,1.0,Offer,EUR,https://zjisin.bandcamp.com/track/seele-im-ges...,NaN,NaN,NaN,NaN
4,https://backtrack.bandcamp.com/album/the-08-demo,"[{'name': 'The '08 Demo', 'url': 'https://back...",https://f4.bcbits.com/img/a1011500902_10.jpg,5.0,"New York, Punk, hardcore, hardcore punk, new y...",11 Apr 2019 01:52:49 GMT,https://backtrack.bandcamp.com/album/the-08-demo,MusicAlbum,The '08 Demo,11 Apr 2019 01:52:49 GMT,https://schema.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://f4.bcbits.com/img/0016064644_10.jpg,https://bandcamp.com/tag/punk,https:

In [4]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 43 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   _id                                 100000 non-null  object 
 1   name                                100000 non-null  object 
 2   albumRelease                        56110 non-null   object 
 3   @id                                 100000 non-null  object 
 4   numTracks                           56110 non-null   float64
 5   datePublished                       100000 non-null  object 
 6   @context                            100000 non-null  object 
 7   @type                               100000 non-null  object 
 8   image                               99983 non-null   object 
 9   description                         45913 non-null   object 
 10  dateModified                        100000 non-null  object 
 11  keywords                   

In [17]:
_df = df.drop(columns=["albumRelease", "@type", "image", "@id", "@context", "@graph"])

In [22]:
_df.dtypes

_id                                    object
numTracks                             float64
keywords                               object
datePublished                          object
name                                   object
dateModified                           object
comment                                object
description                            object
inAlbum                               float64
offers                                float64
duration_secs                         float64
url                                    object
duration                               object
recordingOf                           float64
isrcCode                               object
byArtist_image                         object
byArtist_genre                         object
byArtist_@id                           object
byArtist_@type                         object
byArtist_sameAs                        object
byArtist_name                          object
byArtist_description              

In [12]:
file = "/Users/reneboygarcia/Downloads/test3.parquet"
_df.to_parquet(file, compression="snappy")

In [14]:
df_parquet.shape

(100000, 40)

In [13]:
parquet_file = "/Users/reneboygarcia/Downloads/test.parquet"
df_parquet = pd.read_parquet(parquet_file)
df_parquet.head()

,_id,name,@id,numTracks,datePublished,@context,description,dateModified,keywords,duration_secs,offers,inAlbum,duration,url,isrcCode,recordingOf,comment,byArtist_name,byArtist_@id,byArtist_sameAs,byArtist_image,byArtist_@type,byArtist_description,byArtist_genre,track_itemListElement,track_numberOfItems,track_@type,track,offers_@type,offers_priceSpecification_minPrice,offers_price,offers_availability,offers_priceCurrency,offers_url,inAlbum_@id,inAlbum_@type,inAlbum_name,recordingOf_lyrics_@type,recordingOf_lyrics_text,recordingOf_@type
0,https://laeela.bandcamp.com/album/gratitude,GRATITUDE,https://laeela.bandcamp.com/album/gratitude,9.0,20 May 2019 22:15:58 GMT,https://schema.org,"Deep in the heart of Pacha Mama, the mother of...",06 Aug 2020 10:54:59 GMT,"Devotional, Kundalini Yoga, Medicine, mantra, ...",NaN,NaN,NaN,None,None,None,NaN,None,LAEELA,https://laeela.bandcamp.com,[http://www.laeela.com],https://f4.bcbits.com/img/0020448180_10.jpg,MusicGroup,LAEELA are Neelam and David Mages. When they m...,https://bandcamp.com/tag/devotional,"[{'@type': 'ListItem', 'item': {'@id': 'https:...",9.0,ItemList,NaN,None,NaN,NaN,None,None,None,None,None,None,None,None,None
1,https://goodforyourecords.bandcamp.com/album/l...,Loving You,https://goodforyourecords.bandcamp.com/album/l...,4.0,24 Jul 2019 07:51:28 GMT,https://schema.org,None,24 Jul 2019 07:51:28 GMT,Los Angeles,NaN,NaN,NaN,None,None,None,NaN,None,Kenny Summit & Director's Cut feat. Yasmeen,https://goodforyourecords.bandcamp.com,None,https://f4.bcbits.com/img/0008358174_10.jpg,MusicGroup,None,None,"[{'@type': 'ListItem', 'item': {'@id': 'https:...",4.0,ItemList,NaN,None,NaN,NaN,None,None,None,None,None,None,None,None,None
2,https://alipowers.bandcamp.com/track/sayonara,sayonara,https://alipowers.bandcamp.com/track/sayonara,NaN,24 Jul 2011 14:42:19 GMT,https://schema.org,None,25 Jul 2011 00:11:55 GMT,"Alternative, multiple genres to follow, Derby",277.188,NaN,NaN,P00H04M37S,https://alipowers.bandcamp.com/track/sayonara,None,NaN,None,ali pow3rs,https://alipowers.bandcamp.com,[http://www.alipowers.com],https://f4.bcbits.com/img/0003415976_10.jpg,MusicGroup,a cat watching a butterfly,https://bandcamp.com/tag/alternative,None,NaN,None,NaN,Offer,0.0,0.0,OnlineOnly,GBP,https://alipowers.bandcamp.com/track/sayonara#buy,https://alipowers.bandcamp.com/album/grip,MusicAlbum,grip,None,None,None
3,https://goodkarmarecords.bandcamp.com/track/li...,Like in the 80`s,https://goodkarmarecords.bandcamp.com/track/li...,NaN,23 Dec 2012 20:14:48 GMT,https://schema.org,None,23 Dec 2012 20:14:47 GMT,"Electronic, ambient, electronica, experimental...",447.000,NaN,NaN,P00H07M27S,https://goodkarmarecords.bandcamp.com/track/li...,None,NaN,None,Disappointed Lawyers,https://goodkarmarecords.bandcamp.com,[http://www.goodkarma.ru],None,MusicGroup,None,https://bandcamp.com/tag/electronic,None,NaN,None,NaN,Offer,1.0,1.0,OnlineOnly,EUR,https://goodkarmarecords.bandcamp.com/track/li...,https://goodkarmarecords.bandcamp.com/album/un...,MusicAlbum,Uno The Lonely,None,None,None
4,https://monalisatwins.bandcamp.com/track/one-m...,One More Time,https://monalisatwins.bandcamp.com/track/one-m...,NaN,14 Jul 2013 23:33:04 GMT,https://schema.org,"""One More Time"" from the album ""When We're Tog...",14 Jul 2013 21:33:47 GMT,"60s Rock & Roll, 60s folk-rock, 60s pop-rock, ...",180.245,NaN,NaN,P00H03M00S,https://monalisatwins.bandcamp.com/track/one-m...,ATWB60700033,NaN,None,MonaLisa Twins,https://monalisatwins.bandcamp.com,"[http://www.monalisa-twins.com, http://www.you...",https://f4.bcbits.com/img/0011402209_10.jpg,MusicGroup,The MonaLisa Twins are one of the very few mod...,https://bandcamp.com/tag/pop,None,NaN,None,NaN,Offer,0.0,0.0,OnlineOnly,GBP,https://monalisatwins.bandcamp.com/track/one-m...,https://monalisatwins.bandcamp.com/album/when-...,MusicAlbum,When We're Together,CreativeWork,"There he stands,\r\nLooking in my direction\r\...",MusicComposition


In [67]:
# Seq 2-Define a function to tweak the data frame
def tweak_df(df: pd.DataFrame) -> pd.DataFrame:
    print(f"Number of rows: {df.shape[0]}")
    df_ = df.drop(columns=["albumRelease", "@type", "image", "@id", "@context"])
    return df_

In [87]:
filename = 'bandcamp/albums-json/albums-full-info-3.json'
_file_name = filename.split("/")[-1].split(".")[0]
_file_name

'albums-full-info-3'

In [68]:
# Seq 3-Define a function to set a path for GCS storage and for local file
def write_local(df: pd.DataFrame, filename: str) -> Path:
    directory = Path("bandcamp")
    _file_name = filename.split(".")[0]
    path_name = directory / f"{_file_name}.parquet"
    try:
        os.makedirs(directory)
        df.to_parquet(path_name, compression="snappy")
    except OSError as error:
        print(error)
    return path_name

In [69]:
# Seq 4-Define a function to upload local file to GCS Bucket
def write_to_gcs(path: Path) -> None:
    gcs_block = GcsBucket.load("prefect-gcs-block-bandcamp")
    gcs_block.upload_from_path(from_path=path, to_path=path)
    print("Hooray, we uploaded a huge file in GCS")
    return

In [70]:
# Seq 5-Delete local file and its directory
def duduplicate(path: Path) -> None:
    try:
        path.unlink()
        full_path = path.resolve()
        full_path.parent.rmdir()
        print("Successfully deleted directory and its files")
    except OSError as error:
        print(f"Unable to find directory: {error}")

In [71]:
# Define ETL from web to gcs:
def etl_web_to_gcs(file: str):
    # Seq 1 -Read file
    df = read_df(file)
    # Seq 2 -Tweak df
    df_ = tweak_df(df)
    # Seq 3 -Set a path this will be use to convert file to parquet
    path_file = write_local(df, file)
    # Seq 4-Upload local file to GCS Bucket
    write_to_gcs(path_file)
    # Seq 5- Remove duplicate
    duduplicate(path_file)

In [79]:
# Define download progress hook
def download_progress_hook(block_num, block_size, total_size, progress_bar=None):
    if not progress_bar:
        progress_bar = tqdm(total=total_size, unit="B", unit_scale=True)
    downloaded = block_num * block_size
    progress_bar.update(downloaded - progress_bar.n)
    if downloaded >= total_size:
        progress_bar.close()
    return progress_bar

In [73]:
# Seq 0 -Download file folder from web
def fetch_data(url:str):
    folder_name = url.split("/")[-1].split("?")[0]
    file_folder = urlretrieve(url, folder_name, reporthook=download_progress_hook)
    if folder_name.endswith(".zip"):
        zip_file = ZipFile(folder_name)
        folder_name_ = os.path.commonprefix(zip_file.namelist()).strip("/")
        zip_file.extractall()
        print(f"Download Complete..extracted zip file")
        print(f"Extracted folder path: {folder_name_}")
        return folder_name_
    print(f"Download Complete..")
    return file_folder

In [74]:
# Define a parent ETL to download the files
progress_bar = None
def elt_parent_web_gcs():
    # Parameters
    dataset_url = "https://www.dropbox.com/s/a1kl5e35j4o53mz/bandcamp-items-json.zip?dl=1"

    # Execution
    # Seq 0 -Download file folder from web
    file_folder = fetch_data(dataset_url)
    # Loop through the files then run etl_web_to_gcs
    print("Running etl_web_to_gcs...this will take sometime..grab some coffee or tea")
    for file in os.listdir(file_folder)[:1]:
        if file.endswith(".json"):
            file_path = os.path.join(file_folder, file)
            print(f"Running: {file}")
            etl_web_to_gcs(file_path)
            print(f"Done uploading {file} to GCS")
    print("All files are Uploaded")

In [ ]:
# dataset_url = "https://www.dropbox.com/s/a1kl5e35j4o53mz/bandcamp-items-json.zip?dl=1"
dataset_url_2 = "https://www.dropbox.com/s/wd38q80el16i19q/1000000-bandcamp-sales.zip?dl=1"
file_folder = fetch_data(dataset_url_2)
for file in os.listdir(file_folder):
    if file.endswith(".json"):
        print(file)

In [76]:
# print(os.listdir(file_folder))
# print(os.listdir("albums-json")[0])

In [ ]:
if __name__ == "__main__":
    elt_parent_web_gcs()